In [ ]:
import pprint
from sklearn.cluster import KMeans

def clustering(num_clusters, data,data_labels):
    mu = {}
    sigma = {}
    kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(data)
    l = [data_labels[np.where(kmeans.labels_ == i)] for i in range(num_clusters\
)]

    #pprint.pprint(l)
    #sigma_0 = {i: np.std(data[np.where(kmeans.labels_ == i)], axis=0) for i in
     #          range(num_clusters)}
    for l in range(0, num_clusters):
        mu['cluster_' + str(l)] = kmeans.cluster_centers_[
            l] 
    
    sigma = 10.

    return mu, sigma

In [ ]:
def random_selection(num_clusters, data,data_labels):
    rand_idx = np.random.randint( 0, data.shape[0]-1, size=num_clusters)
    mu = {}
    for l in range(0, num_clusters):
        var = data[rand_idx[l]] 
        mu['cluster_' + str(l)] =  var#tf.Variable(var,dtype=tf.float32,trainable=True)
    sigma = .01
   
    return mu,sigma

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import datasets, layers, models
#from networks.dense_net import DenseNet
from six.moves import xrange
import pprint
from scipy import special
import tensorflow_probability as tfp
from sklearn import preprocessing




def Train_and_Test_RBF(X_train,X_test,y_train,y_test,Y_train,mu,sigma,num_classes):
        '''
        this method will define the network and train the model
        ''' 
        batch_size = 100# X_test.shape[0]
        
        tfd = tfp.distributions
        K = 10
        runs = 30
        tf.compat.v1.disable_eager_execution()
        #mu, sigma = random_selection(K,X_train,Y_train)
        
        x = tf.compat.v1.placeholder(tf.float32, shape=[None, X_train.shape[1]])
        y = tf.compat.v1.placeholder(tf.float32, shape=[None, y_test.shape[1]])
        #sqrt((r/self.epsilon)**2 + 1)
        m_list = list()
        for k in range(K):
            key = 'cluster_'+str(k)
            # this calculates the single point activation as in my paper
            v =  tf.pow(tf.square(x-mu[key])/(tf.square(sigma)+1),.5) # tf.pow(tf.reduce_sum(tf.square(x-mu[key],axis=1)/(tf.square(sigma)+1),.5)
            m_list.append(v)
        #updated_x = tf.transpose(m_list) # Incase the sum is reduced (not  single-point activation)
        
        updated_x = tf.reduce_mean(m_list,axis=0)
        
    

        output_layer = tf.keras.Sequential([
                    layers.Dense(400, activation=tf.nn.sigmoid,name="e_1"),
                    layers.Dense(250, activation=tf.nn.sigmoid,name="e_2"),
                    layers.Dense(num_classes, activation=tf.nn.sigmoid,name="e_3")      
            

                ])
       
    
        output_result = output_layer(updated_x)

        
        cost = tf.reduce_mean(tf.reduce_sum(tf.square(y- output_result),axis=1))#tf.nn.softmax_cross_entropy_with_logits(y, output_result))
        optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=0.0001).minimize(cost)
        

        n_minibatches = int(X_train.shape[0] / batch_size)
        print("Number of minibatches: ", n_minibatches)

        sess = tf.compat.v1.InteractiveSession()
        init = tf.compat.v1.global_variables_initializer()
        sess.run(init)
        for epoch in range(runs):
                    pbar = tf.keras.utils.Progbar(n_minibatches)
                    for i in range(n_minibatches):
                        x_batch = X_train[i * batch_size:(i + 1) * batch_size] 
                        y_batch = y_train[i * batch_size:(i + 1) * batch_size] 
                        cost_, _,output_print,updated_print = sess.run((cost, optimizer, output_result,v), feed_dict={x: x_batch,y:y_batch})
                    if epoch%5 == 0: 
                        
                        pbar.add(1, [("cost", cost_),( "epoch",epoch)])
                        print(np.max(updated_print))
                        
        
        z_test =  sess.run(output_result, feed_dict={x: X_test,y:y_test})
        
        sess.close()
        return z_test
                        


In [ ]:

import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn import preprocessing
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

data = pd.read_csv("data/lung.csv")
print("data shape before split :", data.shape)

#...................................................................................... Normalization
data_normalized = preprocessing.scale(data.values[:,:-1])
y = data.values[:,-1] 
_,num_classes = np.unique(y,return_index = True)
print(num_classes)
data_normalized = np.hstack((data_normalized,y[:,np.newaxis]))



#.....................................................................................splitting into 10-folds
kf = KFold(n_splits=10,shuffle=True) # Define the split - into 10 folds 
kf.get_n_splits(data_normalized) # returns the number of splitting iterations in the cross-validator
#print(kf) 

#.....................................................................................start_training
RF_score_cumm,SMO_score_cumm,NB_score_cumm,LR_score_cumm = 0.,0.,0.,0.
VAE_RF_score_cumm,VAE_SMO_score_cumm,VAE_NB_score_cumm,VAE_LR_score_cumm = 0.,0.,0.,0.
GMV_RF_score_cumm,GMV_SMO_score_cumm,GMV_NB_score_cumm,GMV_LR_score_cumm = 0.,0.,0.,0.

K = 30 # number of clusters
for train_index, test_index in kf.split(data_normalized):
     print("########################## fold #################")
     #print("TRAIN:", train_index, "TEST:", test_index)
     X_train, X_test = data_normalized[train_index,:-1], data_normalized[test_index,:-1]
     y_train, y_test = data_normalized[train_index,-1], data_normalized[test_index,-1]
    
     enc = OneHotEncoder(handle_unknown='ignore')
     enc.fit(y_train[:,np.newaxis])
     Y_train = enc.transform(y_train[:,np.newaxis]).toarray()
  

     Y_test = enc.transform(y_test[:,np.newaxis]).toarray()
    
     mu, sigma = random_selection(K,X_train,Y_train)
     
     # call the NN-model
     test_class = Train_and_Test_RBF(X_train,X_test,Y_train,Y_test,y_train,mu,sigma,num_classes.shape[0])
     print(test_class)





        
        

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn import preprocessing
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn import metrics

(X_train, y_train), (X_test, y_test) = mnist.load_data()
print(" Y train shape : ", y_train)
enc = OneHotEncoder(handle_unknown='ignore')
     
enc.fit(y_train[:,np.newaxis])
Y_train = enc.transform(y_train[:,np.newaxis]).toarray()
#print(Y_train[0:10,:])


Y_test = enc.transform(y_test[:,np.newaxis]).toarray()
     
     # call the NN-model
mu, sigma = random_selection(30,X_train.reshape((X_train.shape[0],28*28))/255.,Y_train)
test_class = Train_and_Test_RBF(X_train.reshape((X_train.shape[0],28*28))/255., 
                                X_test.reshape((X_test.shape[0],28*28))/255.,
                                Y_train, Y_test,y_train,mu,sigma,10)
#print(np.argmax(test_class,axis=1)[0:20])
#print(np.argmax(Y_test,axis=1)[0:20])
f_report = metrics.accuracy_score(np.argmax(Y_test,axis=1),np.argmax(test_class,axis=1), normalize=True)
print("f-score : ", f_report)
